<a href="https://colab.research.google.com/github/DevangVamja/probable-1/blob/main/fraud_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Code to read csv file into colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [3]:

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
#2.1 Get the file
downloaded = drive.CreateFile({'id':'12LDbiRh_SM2b3ym0hO_1RSJpS59OcBO8'}) # replace the id with id of file you want to access
downloaded.GetContentFile('database.csv')

In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten,Dense,Dropout,BatchNormalization
from tensorflow.keras.layers import Conv1D,MaxPool1D
from tensorflow.keras.optimizers import Adam

In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from  sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [7]:
data=pd.read_csv('database.csv')
data.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [8]:
data.shape

(6362620, 11)

In [9]:
data.isnull().sum()

step              0
type              0
amount            0
nameOrig          0
oldbalanceOrg     0
newbalanceOrig    0
nameDest          0
oldbalanceDest    0
newbalanceDest    0
isFraud           0
isFlaggedFraud    0
dtype: int64

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362620 entries, 0 to 6362619
Data columns (total 11 columns):
 #   Column          Dtype  
---  ------          -----  
 0   step            int64  
 1   type            object 
 2   amount          float64
 3   nameOrig        object 
 4   oldbalanceOrg   float64
 5   newbalanceOrig  float64
 6   nameDest        object 
 7   oldbalanceDest  float64
 8   newbalanceDest  float64
 9   isFraud         int64  
 10  isFlaggedFraud  int64  
dtypes: float64(5), int64(3), object(3)
memory usage: 534.0+ MB


In [11]:
data['isFraud'].value_counts()

0    6354407
1       8213
Name: isFraud, dtype: int64

balance dataset


In [12]:
from statsmodels.tools import categorical

# 1. Keep only interested transaction type ('TRANSFER', 'CASH_OUT')
# 2. Drop some columns
# 3. Convert categorical variables to numeric variable
tmp = data.loc[(data['type'].isin(['TRANSFER', 'CASH_OUT'])),:]
tmp = tmp.reset_index(drop=True)
a = np.array(tmp['type'])
b = categorical(a, drop=True)
tmp['type_num'] = b.argmax(1)
tmp.drop(['step', 'nameOrig', 'nameDest', 'isFlaggedFraud','type'], axis=1, inplace=True)

tmp

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,type_num
0,181.00,181.00,0.0,0.00,0.00,1,1
1,181.00,181.00,0.0,21182.00,0.00,1,0
2,229133.94,15325.00,0.0,5083.00,51513.44,0,0
3,215310.30,705.00,0.0,22425.00,0.00,0,1
4,311685.89,10835.00,0.0,6267.00,2719172.89,0,1
...,...,...,...,...,...,...,...
2770404,339682.13,339682.13,0.0,0.00,339682.13,1,0
2770405,6311409.28,6311409.28,0.0,0.00,0.00,1,1
2770406,6311409.28,6311409.28,0.0,68488.84,6379898.11,1,0
2770407,850002.52,850002.52,0.0,0.00,0.00,1,1


In [13]:
non_fraud=tmp[tmp['isFraud']==0]
fraud=tmp[tmp['isFraud']==1]

In [14]:
non_fraud.shape,fraud.shape

((2762196, 7), (8213, 7))

In [15]:
non_fraud=non_fraud.sample(fraud.shape[0])
non_fraud.shape

(8213, 7)

In [16]:
tmp=fraud.append(non_fraud,ignore_index=True)
tmp

,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,type_num
0,181.00,181.0,0.00,0.00,0.00,1,1
1,181.00,181.0,0.00,21182.00,0.00,1,0
2,2806.00,2806.0,0.00,0.00,0.00,1,1
3,2806.00,2806.0,0.00,26202.00,0.00,1,0
4,20128.00,20128.0,0.00,0.00,0.00,1,1
...,...,...,...,...,...,...,...
16421,500674.23,9078.0,0.00,229082.15,729756.37,0,0
16422,228901.27,5086.0,0.00,1306589.74,1535491.02,0,0
16423,239962.12,60909.0,0.00,3545138.67,3785100.79,0,0
16424,95156.44,103154.0,7997.56,954.00,2340997.58,0,0


In [17]:
tmp['isFraud'].value_counts()

1    8213
0    8213
Name: isFraud, dtype: int64

In [18]:
x=tmp.drop('isFraud',axis=1)
y=tmp['isFraud']

In [19]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0,stratify=y)
x_train.shape,x_test.shape

((13140, 6), (3286, 6))

In [20]:
scaler=StandardScaler()
x_train=scaler.fit_transform(x_train)
x_test=scaler.transform(x_test)

In [21]:
y_train=y_train.to_numpy()
y_test=y_test.to_numpy()


In [22]:
x_train=x_train.reshape(x_train.shape[0],x_train.shape[1],1)
x_test=x_test.reshape(x_test.shape[0],x_test.shape[1],1)

In [23]:
x_train.shape,x_test.shape

((13140, 6, 1), (3286, 6, 1))

model


In [24]:
epochs=25
model=Sequential()
model.add(Conv1D(32,2,activation='relu',input_shape=x_train[0].shape))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Conv1D(64,2,activation='sigmoid'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(64,activation='sigmoid'))
model.add(Dropout(0.5))

model.add(Dense(1,activation='sigmoid'))


In [25]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 5, 32)             96        
_________________________________________________________________
batch_normalization (BatchNo (None, 5, 32)             128       
_________________________________________________________________
dropout (Dropout)            (None, 5, 32)             0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 4, 64)             4160      
_________________________________________________________________
batch_normalization_1 (Batch (None, 4, 64)             256       
_________________________________________________________________
dropout_1 (Dropout)          (None, 4, 64)             0         
_________________________________________________________________
flatten (Flatten)            (None, 256)               0

In [26]:
model.compile(optimizer=Adam(lr=0.001),loss=('binary_crossentropy'),metrics=['accuracy'])

In [27]:
history=model.fit(x_train,y_train,epochs=epochs,validation_data=(x_test,y_test),verbose=1)

Epoch 1/25
411/411 [==============================] - 1s 3ms/step - loss: 0.3193 - accuracy: 0.8613 - val_loss: 0.2852 - val_accuracy: 0.8981
Epoch 2/25
411/411 [==============================] - 1s 3ms/step - loss: 0.2370 - accuracy: 0.9020 - val_loss: 0.2212 - val_accuracy: 0.9051
Epoch 3/25
411/411 [==============================] - 1s 3ms/step - loss: 0.2109 - accuracy: 0.9088 - val_loss: 0.1847 - val_accuracy: 0.9136
Epoch 4/25
411/411 [==============================] - 1s 3ms/step - loss: 0.2040 - accuracy: 0.9135 - val_loss: 0.1649 - val_accuracy: 0.9224
Epoch 5/25
411/411 [==============================] - 1s 3ms/step - loss: 0.1898 - accuracy: 0.9177 - val_loss: 0.1805 - val_accuracy: 0.9124
Epoch 6/25
411/411 [==============================] - 1s 3ms/step - loss: 0.1805 - accuracy: 0.9218 - val_loss: 0.1661 - val_accuracy: 0.9209
Epoch 7/25
411/411 [==============================] - 1s 3ms/step - loss: 0.1772 - accuracy: 0.9253 - val_loss: 0.1777 - val_accuracy: 0.9175
Epoch 